#Description


In this notebook we dealt with the mixed_pataset resulting from the spliting of the original dataset previously to deal with each similar cases in the same way
This notebook focuses on processing and filtering activity suggestions and commercial activities from a mixed-language dataset (Arabic and French where users enetered both french and arabic languages in the same cell )and matching them with an official activity list. The workflow includes:

🔹 Data Loading & Preprocessing

Load the mixed dataset and the official activity list.
Detect the dominant language for each activity.
Normalize and clean text for both Arabic and French.

🔹 Embedding Generation

multiple methodologies have been iplemented:

Generate text embeddings using multilingual transformer models:

*   French: paraphrase-multilingual-MiniLM-L12-v2
*   Arabic: CAMeL-BERT.

but the final version includes:

*   Generate text embeddings using Alibaba-NLP/gte-multilingual-base for both french and arabic.

🔹 Activity Matching

Compute cosine similarity between embeddings.
Assign the best-matching activity if similarity exceeds a threshold.

🔹 Results & Export

Save matched activities to a separate file.
Save unmatched activities for further review.
This process ensures a structured and automated approach to classifying suggested activities while handling multilingual data efficiently.

🔹 Filtering Traditional Commercial Activities:

A predefined list of common commercial activities is compared using semantic similarity.
Activities classified as traditional commercial (e.g., retail, food services, import/export) are excluded.
E-commerce & digital intermediary activities are retained for further review.
Final Output & Categorization:

The system generates three output files:

*   Matched Activities → Successfully mapped to official records.
*   Not Matched Activities → Unmatched activities for further review.
*   Filtered Non-Commercial Activities → Excludes traditional commercial
activities while keeping innovative ones.

This version ensures high accuracy, faster processing, and refined classification, making it an efficient solution for structured activity categorization.



In [9]:
!pip install pandas langdetect openpyxl

#MIXED DATASET

In [10]:
import pandas as pd

In [11]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## preprocessing

In [12]:
file_path = '/content/drive/My Drive/DATASET SPLIT/proposition_MIXED.xlsx'  # Update this path
df_mixed = pd.read_excel(file_path)

# Step 3: Display the first few rows of the DataFrame
print(df_mixed.head())

   code_pro   wilaya                                 field  \
0  NFL77K67  Mascara  Services de loisirs et de récréation   
1  YEY4MTKN  Ouargla                Services à la personne   
2  DKVFC9X8  Algiers                                   NaN   
3  7YCMDUFE  Algiers                Services à la personne   
4  AXPQB8GL    Msila                Prestations à domicile   

                activity                                        description  
0         كهرباء معمارية                                                NaN  
1                  نجارة   Fabrication de cuisines et chambres modernes MDF  
2               TOURISME                                                NaN  
3           Garde malade                                                NaN  
4  كل أعمال الجبس الفنية  الأشغال الصغيرة للبناء والدهن \nحرفي جباس  \nك...  


In [13]:
df_mixed

,code_pro,wilaya,field,activity,description
0,NFL77K67,Mascara,Services de loisirs et de récréation,كهرباء معمارية,NaN
1,YEY4MTKN,Ouargla,Services à la personne,نجارة,Fabrication de cuisines et chambres modernes MDF
2,DKVFC9X8,Algiers,NaN,TOURISME,NaN
3,7YCMDUFE,Algiers,Services à la personne,Garde malade,NaN
4,AXPQB8GL,Msila,Prestations à domicile,كل أعمال الجبس الفنية,الأشغال الصغيرة للبناء والدهن \nحرفي جباس \nك...
...,...,...,...,...,...
3403,1DF55VKL,Batna,Services aux entreprises,Menuiseries aluminium et pvc,نجارة الألمنيوم والمواد البلاستيكية
3404,SSFAZK3P,Sidi Bel Abbes,Services à la personne,صانع عصائر و مشروبات,صانع عصائر و مشروبات في المطاعم و المقاهي
3405,WG8K44WD,Guelma,Services aux entreprises,مكتب دراسات تقنية هندسة مدنية,دراسات تقنية في مجال الهندسة المدينة\n-متابعة ...
3406,MBQNVBK2,Bouira,"Conseil, Expertise et formation","Expertise en Génie Civil, Ingénieur en Génie C...",NaN


In [14]:
# Install necessary libraries
!pip install langdetect
!pip install sentence-transformers

In [15]:
!pip install unidecode


In [8]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer
import numpy as np
import re
from unidecode import unidecode
from sklearn.metrics.pairwise import cosine_similarity
from langdetect import detect

# using camel bert


In [ ]:
# Load separate models for French and Arabic
model_fr = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')  # Example model for French
arabic_tokenizer = AutoTokenizer.from_pretrained("CAMeL-Lab/bert-base-arabic-camelbert-msa")
arabic_model = AutoModel.from_pretrained("CAMeL-Lab/bert-base-arabic-camelbert-msa")

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/86.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/468 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/305k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

In [ ]:
def generate_arabic_embedding(text):
    tokens = arabic_tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = arabic_model(**tokens)
    cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze().numpy()  # Use CLS token representation
    return cls_embedding

    # Step 2: Concatenate activity and description
df_mixed['combined_text'] = df_mixed['activity'] + " " + df_mixed['description']

# Step 3: Function to detect the dominant language
def detect_dominant_language(text):
    languages = []
    try:
        for word in text.split():
            lang = detect(word)
            languages.append(lang)
    except:
        return None
    # Return the most common language detected
    return max(set(languages), key=languages.count)

# Apply language detection
df_mixed['dominant_language'] = df_mixed['combined_text'].apply(detect_dominant_language)


model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

In [ ]:

# Step 4: Preprocessing function
# Function to normalize Arabic text
def normalize_arabic(text):
    text = re.sub(r'[\u064B-\u065F]', '', text)  # Remove Arabic diacritics
    text = text.replace("أ", "ا").replace("إ", "ا").replace("آ", "ا")  # Normalize Alef variations
    text = text.replace("ة", "ه")  # Normalize Ta Marbuta
    text = text.replace("ى", "ي")  # Normalize Alef Maksura
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    return text

# Modified normalization function
def normalize_text(text, lang):
    if not isinstance(text, str) or text.strip() == "":
        return ""
    text = text.lower().strip()
    if lang == "ar":
        text = normalize_arabic(text)  # Apply Arabic-specific preprocessing
    else:
        text = unidecode(text)  # Use Unidecode for French text
    text = re.sub(r'\d+', '', text)  # Remove digits
    return text


# Apply preprocessing based on detected language
df_mixed['processed_text'] = df_mixed.apply(lambda row: normalize_text(row['combined_text'], row['dominant_language']), axis=1)

# Step 5: Generate embeddings for the processed text using the appropriate model
def generate_embeddings(row):
    if row['dominant_language'] == 'fr':
        return model_fr.encode(row['processed_text'])  # Use SentenceTransformer for French
    elif row['dominant_language'] == 'ar':
        return generate_arabic_embedding(row['processed_text'])  # Use CAMeL-BERT for Arabic
    return None

df_mixed['embeddings'] = df_mixed.apply(generate_embeddings, axis=1)




In [ ]:
df_activities = pd.read_excel('/content/drive/My Drive/hackathon1/activities-hackathon/activity_list.xlsx')
 #Step 6: Process activity list based on language
# Apply the same for official activity lists
df_activities['processed_french'] = df_activities['name_activity'].apply(lambda x: normalize_text(x, 'fr'))
df_activities['processed_arabic'] = df_activities['ar_name_activity'].apply(lambda x: normalize_text(x, 'ar'))

# Generate embeddings for both French and Arabic official activities using the appropriate model
df_activities['french_embedding'] = df_activities['processed_french'].apply(lambda x: model_fr.encode(x))
df_activities['arabic_embedding'] = df_activities['processed_arabic'].apply(lambda x: generate_arabic_embedding(x))  # Use the defined function

# Step 7: Matching function based on dominant language
# Step 7: Matching function based on dominant language
def find_match(row):
    if row['dominant_language'] == 'fr':
        similarities = df_activities['french_embedding'].apply(lambda x: cosine_similarity([row['embeddings']], [x])[0][0])
        max_similarity = similarities.max()
        if max_similarity >= 0.6:
            best_match = df_activities.iloc[similarities.idxmax()]
            return best_match['name_activity'], best_match['code_activity']

    elif row['dominant_language'] == 'ar':
        similarities = df_activities['arabic_embedding'].apply(lambda x: cosine_similarity([row['embeddings']], [x])[0][0])
        max_similarity = similarities.max()
        if max_similarity >= 0.6:
            best_match = df_activities.iloc[similarities.idxmax()]
            return best_match['ar_name_activity'], best_match['code_activity']

    return None, None

# Apply matching function
df_mixed[['matched_activity', 'matched_code']] = df_mixed.apply(lambda row: pd.Series(find_match(row)), axis=1)

# Step 8: Save the results
df_mixed.to_excel('/content/drive/My Drive/hackathon1/matched_activities.xlsx', index=False)

# Display some results
print(df_mixed[['combined_text', 'dominant_language', 'processed_text', 'matched_activity', 'matched_code']].head())


In [ ]:
# Filter rows where a match was found
df_matched = df_mixed[df_mixed['matched_activity'].notna() & df_mixed['matched_code'].notna()]

# Print matched rows
print(df_matched[['combined_text', 'dominant_language', 'matched_activity', 'matched_code']])

# Filter matched rows
df_matched = df_mixed[df_mixed['matched_activity'].notna() & df_mixed['matched_code'].notna()]

# Save to a separate file
matched_file_path = "/content/drive/My Drive/hackathon1/matched_activities_only.xlsx"
df_matched.to_excel(matched_file_path, index=False)

print(f"Matched activities saved to: {matched_file_path}")

# Filter rows where NO match was found
df_not_matched = df_mixed[df_mixed['matched_activity'].isna() | df_mixed['matched_code'].isna()]

# Save unmatched rows to a separate file
not_matched_file_path = "/content/drive/My Drive/hackathon1/not_matched_activities.xlsx"
df_not_matched.to_excel(not_matched_file_path, index=False)

print(f"Unmatched activities saved to: {not_matched_file_path}")

# using alibaba

In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer
import numpy as np
import re
from unidecode import unidecode
from sklearn.metrics.pairwise import cosine_similarity
from langdetect import detect

# Load separate models for French and Arabic
model_fr = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')  # Example model for French
model_ar = SentenceTransformer("Alibaba-NLP/gte-multilingual-base", trust_remote_code=True)

# Step 2: Concatenate activity and description
df_mixed['combined_text'] = df_mixed['activity'] + " " + df_mixed['description']

# Step 3: Function to detect the dominant language
def detect_dominant_language(text):
    languages = []
    try:
        for word in text.split():
            lang = detect(word)
            languages.append(lang)
    except:
        return None
    return max(set(languages), key=languages.count)

# Apply language detection
df_mixed['dominant_language'] = df_mixed['combined_text'].apply(detect_dominant_language)

# Step 4: Preprocessing function
def normalize_arabic(text):
    text = re.sub(r'[\u064B-\u065F]', '', text)  # Remove Arabic diacritics
    text = text.replace("أ", "ا").replace("إ", "ا").replace("آ", "ا")  # Normalize Alef variations
    text = text.replace("ة", "ه")  # Normalize Ta Marbuta
    text = text.replace("ى", "ي")  # Normalize Alef Maksura
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    return text

def normalize_text(text, lang):
    if not isinstance(text, str) or text.strip() == "":
        return ""
    text = text.lower().strip()
    if lang == "ar":
        text = normalize_arabic(text)
    else:
        text = unidecode(text)
    text = re.sub(r'\d+', '', text)  # Remove digits
    return text

# Apply preprocessing
df_mixed['processed_text'] = df_mixed.apply(lambda row: normalize_text(row['combined_text'], row['dominant_language']), axis=1)

# Step 5: Generate embeddings for the processed text using the appropriate model
def generate_embeddings(row):
    if row['dominant_language'] == 'fr':
        return model_fr.encode(row['processed_text'])
    elif row['dominant_language'] == 'ar':
        return model_ar.encode(row['processed_text'])  # Using Alibaba GTE model
    return None

df_mixed['embeddings'] = df_mixed.apply(generate_embeddings, axis=1)

# Load official activity list
df_activities = pd.read_excel('/content/drive/My Drive/hackathon1/activities-hackathon/activity_list.xlsx')

df_activities['processed_french'] = df_activities['name_activity'].apply(lambda x: normalize_text(x, 'fr'))
df_activities['processed_arabic'] = df_activities['ar_name_activity'].apply(lambda x: normalize_text(x, 'ar'))

df_activities['french_embedding'] = df_activities['processed_french'].apply(lambda x: model_fr.encode(x))
df_activities['arabic_embedding'] = df_activities['processed_arabic'].apply(lambda x: model_ar.encode(x))



In [ ]:
# Step 7: Matching function
def find_match(row):
    if row['dominant_language'] == 'fr':
        similarities = df_activities['french_embedding'].apply(lambda x: cosine_similarity([row['embeddings']], [x])[0][0])
    elif row['dominant_language'] == 'ar':
        similarities = df_activities['arabic_embedding'].apply(lambda x: cosine_similarity([row['embeddings']], [x])[0][0])
    else:
        return None, None

    max_similarity = similarities.max()
    if max_similarity >= 0.7:
        best_match = df_activities.iloc[similarities.idxmax()]
        return best_match['name_activity'] if row['dominant_language'] == 'fr' else best_match['ar_name_activity'], best_match['code_activity']

    return None, None

# Apply matching function
df_mixed[['matched_activity', 'matched_code']] = df_mixed.apply(lambda row: pd.Series(find_match(row)), axis=1)

# Step 8: Save the results
df_mixed.to_excel('/content/drive/My Drive/hackathon1/matched_activities.xlsx', index=False)

# Display some results
print(df_mixed[['combined_text', 'dominant_language', 'processed_text', 'matched_activity', 'matched_code']].head())

# Filter rows where a match was found
df_matched = df_mixed[df_mixed['matched_activity'].notna() & df_mixed['matched_code'].notna()]

# Print matched rows
print(df_matched[['combined_text', 'dominant_language', 'matched_activity', 'matched_code']])

# Filter matched rows
df_matched = df_mixed[df_mixed['matched_activity'].notna() & df_mixed['matched_code'].notna()]

# Save to a separate file
matched_file_path = "/content/drive/My Drive/hackathon1/matched_activities_only.xlsx"
df_matched.to_excel(matched_file_path, index=False)

print(f"Matched activities saved to: {matched_file_path}")

# Filter rows where NO match was found
df_not_matched = df_mixed[df_mixed['matched_activity'].isna() | df_mixed['matched_code'].isna()]

# Save unmatched rows to a separate file
not_matched_file_path = "/content/drive/My Drive/hackathon1/not_matched_activities.xlsx"
df_not_matched.to_excel(not_matched_file_path, index=False)

print(f"Unmatched activities saved to: {not_matched_file_path}")


                                       combined_text dominant_language  \
0                                                NaN              None   
1  نجارة Fabrication de cuisines et chambres mode...                fr   
2                                                NaN              None   
3                                                NaN              None   
4  كل أعمال الجبس الفنية الأشغال الصغيرة للبناء و...                ar   

                                      processed_text matched_activity  \
0                                                                None   
1  njr@ fabrication de cuisines et chambres moder...             None   
2                                                                None   
3                                                                None   
4  كل اعمال الجبس الفنيه الاشغال الصغيره للبناء و...             None   

   matched_code  
0           NaN  
1           NaN  
2           NaN  
3           NaN  
4           NaN  
        

## using alibaba for whatever the language is


In [ ]:
# Load the multilingual Alibaba model
model = SentenceTransformer("Alibaba-NLP/gte-multilingual-base", trust_remote_code=True)
# Function to normalize Arabic text
def normalize_arabic(text):
    text = re.sub(r'[\u064B-\u065F]', '', text)  # Remove Arabic diacritics
    text = text.replace("أ", "ا").replace("إ", "ا").replace("آ", "ا")  # Normalize Alef variations
    text = text.replace("ة", "ه")  # Normalize Ta Marbuta
    text = text.replace("ى", "ي")  # Normalize Alef Maksura
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    return text

# Function to normalize text (both Arabic & French)
def normalize_text(text):
    if not isinstance(text, str) or text.strip() == "":
        return ""
    text = text.lower().strip()
    if any("\u0600" <= c <= "\u06FF" for c in text):  # Check if Arabic characters exist
        text = normalize_arabic(text)
    else:
        text = unidecode(text)  # Use Unidecode for French
    text = re.sub(r'\d+', '', text)  # Remove digits
    return text

# Apply text normalization
df_mixed['processed_text'] = df_mixed['combined_text'].apply(normalize_text)

# Generate embeddings for all processed text
df_mixed['embeddings'] = df_mixed['processed_text'].apply(lambda x: model.encode(x))

# Load official activity list
df_activities = pd.read_excel('/content/drive/My Drive/hackathon1/activities-hackathon/activity_list.xlsx')

# Normalize official activity names
df_activities['processed_activity'] = df_activities['name_activity'].apply(normalize_text)
df_activities['processed_activity_ar'] = df_activities['ar_name_activity'].apply(normalize_text)

# Generate embeddings for official activities
df_activities['activity_embedding'] = df_activities['processed_activity'].apply(lambda x: model.encode(x))
df_activities['activity_embedding_ar'] = df_activities['processed_activity_ar'].apply(lambda x: model.encode(x))

# Matching function
def find_match(row):
    similarities_fr = df_activities['activity_embedding'].apply(lambda x: cosine_similarity([row['embeddings']], [x])[0][0])
    similarities_ar = df_activities['activity_embedding_ar'].apply(lambda x: cosine_similarity([row['embeddings']], [x])[0][0])

    max_similarity_fr = similarities_fr.max()
    max_similarity_ar = similarities_ar.max()

    # Choose best match from either Arabic or French activities
    if max_similarity_fr >= 0.6 or max_similarity_ar >= 0.6:
        if max_similarity_fr > max_similarity_ar:
            best_match = df_activities.iloc[similarities_fr.idxmax()]
        else:
            best_match = df_activities.iloc[similarities_ar.idxmax()]
        return best_match['name_activity'], best_match['ar_name_activity'], best_match['code_activity']

    return None, None, None

# Apply matching
df_mixed[['matched_activity_fr', 'matched_activity_ar', 'matched_code']] = df_mixed.apply(lambda row: pd.Series(find_match(row)), axis=1)

# Save results
df_mixed.to_excel('/content/drive/My Drive/hackathon1/matched_activities.xlsx', index=False)

print("Matching process completed and saved to Excel.")

# Filter rows where a match was found
df_matched = df_mixed[df_mixed['matched_activity'].notna() & df_mixed['matched_code'].notna()]

# Print matched rows
print(df_matched[['combined_text', 'dominant_language', 'matched_activity', 'matched_code']])

# Filter matched rows
df_matched = df_mixed[df_mixed['matched_activity'].notna() & df_mixed['matched_code'].notna()]

# Save to a separate file
matched_file_path = "/content/drive/My Drive/hackathon1/matched_activities_only.xlsx"
df_matched.to_excel(matched_file_path, index=False)

print(f"Matched activities saved to: {matched_file_path}")

# Filter rows where NO match was found
df_not_matched = df_mixed[df_mixed['matched_activity'].isna() | df_mixed['matched_code'].isna()]

# Save unmatched rows to a separate file
not_matched_file_path = "/content/drive/My Drive/hackathon1/not_matched_activities.xlsx"
df_not_matched.to_excel(not_matched_file_path, index=False)

print(f"Unmatched activities saved to: {not_matched_file_path}")



Some weights of the model checkpoint at Alibaba-NLP/gte-multilingual-base were not used when initializing NewModel: {'classifier.weight', 'classifier.bias'}
- This IS expected if you are initializing NewModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing NewModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## optimized

In [22]:
import pandas as pd
import re
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from unidecode import unidecode

# Load the multilingual Alibaba model
model = SentenceTransformer("Alibaba-NLP/gte-multilingual-base", trust_remote_code=True)

# Function to normalize Arabic text
def normalize_arabic(text):
    text = re.sub(r'[\u064B-\u065F]', '', text)  # Remove Arabic diacritics
    text = text.replace("أ", "ا").replace("إ", "ا").replace("آ", "ا")  # Normalize Alef variations
    text = text.replace("ة", "ه")  # Normalize Ta Marbuta
    text = text.replace("ى", "ي")  # Normalize Alef Maksura
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    return text

# Function to normalize text (both Arabic & French)
def normalize_text(text):
    if not isinstance(text, str) or text.strip() == "":
        return ""
    text = text.lower().strip()
    if any("\u0600" <= c <= "\u06FF" for c in text):  # Check if Arabic characters exist
        text = normalize_arabic(text)
    else:
        text = unidecode(text)  # Use Unidecode for French
    text = re.sub(r'\d+', '', text)  # Remove digits
    return text

# Step 2: Concatenate activity and description
df_mixed['combined_text'] = df_mixed['activity'] + " " + df_mixed['description']

# Apply text normalization
#df_mixed['processed_text'] = df_mixed['combined_text'].apply(normalize_text)

# Generate embeddings for all processed text
df_mixed['embeddings'] = df_mixed['combined_text'].astype(str).apply(lambda x: model.encode(x) if x.strip() else np.zeros(model.get_sentence_embedding_dimension()))

# Load official activity list
df_activities = pd.read_excel('/content/drive/My Drive/hackathon1/activities-hackathon/activity_list.xlsx')

# Normalize official activity names
# df_activities['processed_activity'] = df_activities['name_activity'].apply(normalize_text)
# df_activities['processed_activity_ar'] = df_activities['ar_name_activity'].apply(normalize_text)

# Generate embeddings for official activities
df_activities['activity_embedding'] = df_activities['name_activity'].astype(str).apply(
    lambda x: model.encode(x) if x.strip() else np.zeros(model.get_sentence_embedding_dimension())
)
df_activities['activity_embedding_ar'] = df_activities['ar_name_activity'].astype(str).apply(
    lambda x: model.encode(x) if x.strip() else np.zeros(model.get_sentence_embedding_dimension())
)

# 🚀 Convert embeddings to NumPy arrays for faster similarity search
activity_embeddings_fr = np.stack(df_activities['activity_embedding'].values)
activity_embeddings_ar = np.stack(df_activities['activity_embedding_ar'].values)

# Step 7: Optimized Matching Function
def find_best_match(row):
    # Compute similarity in bulk
    similarities_fr = cosine_similarity([row['embeddings']], activity_embeddings_fr)[0]
    similarities_ar = cosine_similarity([row['embeddings']], activity_embeddings_ar)[0]

    # Combine all similarities
    all_similarities = np.concatenate((similarities_fr, similarities_ar))

    # Get best match
    best_idx = np.argmax(all_similarities)
    best_similarity = all_similarities[best_idx]

    # Debugging output
    print(f"Best index: {best_idx}, Best similarity: {best_similarity}")

    if best_similarity >= 0.8 and best_idx < len(df_activities):
        best_match = df_activities.iloc[best_idx]
        return best_match['name_activity'], best_match['ar_name_activity'], best_match['code_activity']

    # Ensure we return a tuple of three elements
    return None, None, None

# Check if df_mixed and df_activities are not empty
if not df_mixed.empty and not df_activities.empty:
    # Apply the optimized matching function
    results = df_mixed.apply(find_best_match, axis=1)

    # Ensure results is a DataFrame
    df_results = pd.DataFrame(results.tolist(), columns=['matched_activity', 'matched_activity_ar', 'matched_code'])

    # Assign the results back to df_mixed
    df_mixed[['matched_activity', 'matched_activity_ar', 'matched_code']] = df_results

    # Save results
    df_mixed.to_excel('/content/drive/My Drive/hackathon1/matched_activities.xlsx', index=False)
    print("Matching process completed and saved to Excel.")

    # Filter rows where a match was found
    df_matched = df_mixed[df_mixed['matched_activity'].notna() & df_mixed['matched_code'].notna()]

    # Print matched rows
    print(df_matched[['combined_text', 'matched_activity', 'matched_activity_ar', 'matched_code']])

    # Save matched rows to a separate file
    matched_file_path = "/content/drive/My Drive/hackathon1/matched_activities_only.xlsx"
    df_matched.to_excel(matched_file_path, index=False)
    print(f"Matched activities saved to: {matched_file_path}")

    # Filter rows where NO match was found
    df_not_matched = df_mixed[df_mixed['matched_activity'].isna() | df_mixed['matched_code'].isna()]

    # Save unmatched rows to a separate file
    not_matched_file_path = "/content/drive/My Drive/hackathon1/not_matched_activities.xlsx"
    df_not_matched.to_excel(not_matched_file_path, index=False)
    print(f"Unmatched activities saved to: {not_matched_file_path}")
else:
    print("No data to process. Please check the input DataFrames.")


Some weights of the model checkpoint at Alibaba-NLP/gte-multilingual-base were not used when initializing NewModel: {'classifier.weight', 'classifier.bias'}
- This IS expected if you are initializing NewModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing NewModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Best index: 2910, Best similarity: 1.0
Best index: 1617, Best similarity: 0.6400606632232666
Best index: 2910, Best similarity: 1.0
Best index: 2910, Best similarity: 1.0
Best index: 6, Best similarity: 0.683967649936676
Best index: 2910, Best similarity: 1.0
Best index: 2910, Best similarity: 1.0
Best index: 2910, Best similarity: 1.0
Best index: 2910, Best similarity: 1.0
Best index: 2910, Best similarity: 1.0
Best index: 1910, Best similarity: 0.7536624670028687
Best index: 2910, Best similarity: 1.0
Best index: 554, Best similarity: 0.6077630519866943
Best index: 2910, Best similarity: 1.0
Best index: 6, Best similarity: 0.6833605766296387
Best index: 2910, Best similarity: 1.0
Best index: 2910, Best similarity: 1.0
Best index: 2910, Best similarity: 1.0
Best index: 2910, Best similarity: 1.0
Best index: 2843, Best similarity: 0.8458501696586609
Best index: 2910, Best similarity: 1.0
Best index: 2910, Best similarity: 1.0
Best index: 2910, Best similarity: 1.0
Best index: 2910, Bes

## commercial use


In [23]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util

# Load a multilingual model for embeddings
model = SentenceTransformer("Alibaba-NLP/gte-multilingual-base", trust_remote_code=True)

# Lists of traditional commercial activities (to exclude)
traditional_commercial_activities = [
    "restaurant", "vente", "achat", "café", "fast food", "boucherie", "poissonnerie", "téléphone",
    "bijouterie", "magasin", "boutique", "fournisseur", "détaillant", "grossiste", "revendeur",
    "importateur", "exportateur", "franchise", "vendeur", "commerçant", "négoce", "dépôt vente",
    "magasin alimentaire", "produits alimentaires", "produits de consommation", "liquidation",
    "bazar", "marché", "vente de voitures", "agents commerciaux", "services de livraison",
    "construction", "matériel de construction", "équipement industriel", "téléphonie mobile", "réparation",
    "location", "distribution", "importation", "commerce de détail", "distributeur", "commerçant de proximité",
    "agricole", "produits agricoles", "alimentation générale", "commerçant ambulant", "transports",
    "مطعم", "بيع", "شراء", "مقهى", "وجبات سريعة", "جزارة", "سمك", "هاتف", "مجوهرات", "متجر", "دكان",
    "مورد", "تاجر تجزئة", "تاجر جملة", "بائع", "مستورد", "مُصدر", "امتياز", "تجارة", "بيع بالتجزئة",
    "منتجات غذائية", "منتجات استهلاكية", "تصفية", "بازار", "سوق", "بيع السيارات", "وكلاء تجاريين",
    "خدمات التوصيل", "إنشاء", "معدات البناء", "تجهيزات صناعية", "هواتف محمولة", "صيانة", "إيجار",
    "توزيع", "استيراد", "تجارة التجزئة", "موزع", "تاجر محلي", "زراعي", "منتجات زراعية", "تجارة عامة",
    "تاجر متجول", "نقل"
]

# Allowed e-commerce & intermediary activities (to keep)
ecommerce_intermediary_activities = [
    "e-commerce", "plateforme de vente", "e-shop", "site marchand", "marketplace", "dropshipping",
    "affiliation", "vente en ligne", "webshop", "commerce virtuel", "commerce digital", "fournisseur e-commerce",
    "commerce électronique", "start-up", "entrepreneur digital", "services de paiement", "agent commercial",
    "courtier", "publicité en ligne", "consultant", "services en ligne", "freelance", "plateforme de freelancing",
    "commercialisation", "digital marketing", "plateforme B2B", "réseau de distribution",
    "تجارة إلكترونية", "منصة بيع", "متجر إلكتروني", "موقع تجاري", "سوق إلكتروني", "دروبشيبينغ",
    "تسويق بالعمولة", "متجر على الإنترنت", "تجارة افتراضية", "تجارة رقمية", "مورد تجارة إلكترونية",
    "شركة ناشئة", "ريادي رقمي", "خدمات الدفع", "وكيل تجاري", "سمسار", "إعلانات عبر الإنترنت",
    "مستشار", "خدمات عبر الإنترنت", "مستقل", "منصة عمل مستقل", "تسويق", "تسويق رقمي", "إدارة الحملات",
    "منصة إعلانات", "تسويق بالعمولة", "وسيط", "وكيل مبيعات", "حلول رقمية", "تخزين سحابي", "دعم العملاء",
    "إعلانات رقمية", "وسائل التواصل الاجتماعي", "منصة بي تو بي", "شبكة توزيع"
]

# Encode reference lists
embeddings_commercial = model.encode(traditional_commercial_activities, convert_to_tensor=True)
embeddings_ecommerce = model.encode(ecommerce_intermediary_activities, convert_to_tensor=True)

# Function to determine if an activity is traditional commercial
def classify_activity(activity_text):
    if pd.isna(activity_text):
        return False

    activity_embedding = model.encode(activity_text, convert_to_tensor=True)

    # Compute similarity scores
    similarity_commercial = util.cos_sim(activity_embedding, embeddings_commercial).max().item()
    similarity_ecommerce = util.cos_sim(activity_embedding, embeddings_ecommerce).max().item()

    # Threshold-based filtering
    if similarity_commercial > 0.7 and similarity_commercial > similarity_ecommerce:
        return True  # Exclude (Traditional Commercial)

    return False  # Keep (E-commerce / Valid Activity)

# Apply filtering
df_not_matched['is_traditional_commercial'] = df_not_matched['combined_text'].apply(classify_activity)

# Separate valid and removed activities
df_valid_activities = df_not_matched[df_not_matched['is_traditional_commercial'] == False].drop(columns=['is_traditional_commercial'])
df_removed_activities = df_not_matched[df_not_matched['is_traditional_commercial'] == True].drop(columns=['is_traditional_commercial'])

# Save filtered datasets
valid_activities_file = "/content/drive/My Drive/hackathon1/final_filtered_non_commercial.xlsx"
removed_activities_file = "/content/drive/My Drive/hackathon1/removed_commercial_activities.xlsx"

df_valid_activities.to_excel(valid_activities_file, index=False)
df_removed_activities.to_excel(removed_activities_file, index=False)

print(f"✅ Final filtered non-commercial & e-commerce activities saved to: {valid_activities_file}")
print(f"❌ Removed traditional commercial activities saved to: {removed_activities_file}")


Some weights of the model checkpoint at Alibaba-NLP/gte-multilingual-base were not used when initializing NewModel: {'classifier.weight', 'classifier.bias'}
- This IS expected if you are initializing NewModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing NewModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
<ipython-input-23-059ceebd71ee>:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_not_matched['is_traditional_commercial'] = df_not_matched['comb

✅ Final filtered non-commercial & e-commerce activities saved to: /content/drive/My Drive/hackathon1/final_filtered_non_commercial.xlsx
❌ Removed traditional commercial activities saved to: /content/drive/My Drive/hackathon1/removed_commercial_activities.xlsx


# same optimize but just with activity not combiend with description

In [28]:
import pandas as pd
import re
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from unidecode import unidecode

# Load the multilingual Alibaba model
model = SentenceTransformer("Alibaba-NLP/gte-multilingual-base", trust_remote_code=True)

# Function to normalize Arabic text
def normalize_arabic(text):
    text = re.sub(r'[\u064B-\u065F]', '', text)  # Remove Arabic diacritics
    text = text.replace("أ", "ا").replace("إ", "ا").replace("آ", "ا")  # Normalize Alef variations
    text = text.replace("ة", "ه")  # Normalize Ta Marbuta
    text = text.replace("ى", "ي")  # Normalize Alef Maksura
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    return text

# Function to normalize text (both Arabic & French)
def normalize_text(text):
    if not isinstance(text, str) or text.strip() == "":
        return ""
    text = text.lower().strip()
    if any("\u0600" <= c <= "\u06FF" for c in text):  # Check if Arabic characters exist
        text = normalize_arabic(text)
    else:
        text = unidecode(text)  # Use Unidecode for French
    text = re.sub(r'\d+', '', text)  # Remove digits
    return text


# Apply text normalization
df_mixed['processed_text'] = df_mixed['activity'].apply(normalize_text)

# Generate embeddings for all processed text
df_mixed['embeddings'] = df_mixed['processed_text'].astype(str).apply(lambda x: model.encode(x) if x.strip() else np.zeros(model.get_sentence_embedding_dimension()))

# Load official activity list
df_activities = pd.read_excel('/content/drive/My Drive/hackathon1/activities-hackathon/activity_list.xlsx')

# Normalize official activity names
df_activities['processed_activity'] = df_activities['name_activity'].apply(normalize_text)
df_activities['processed_activity_ar'] = df_activities['ar_name_activity'].apply(normalize_text)

# Generate embeddings for official activities
df_activities['activity_embedding'] = df_activities['processed_activity'].astype(str).apply(
    lambda x: model.encode(x) if x.strip() else np.zeros(model.get_sentence_embedding_dimension())
)
df_activities['activity_embedding_ar'] = df_activities['processed_activity_ar'].astype(str).apply(
    lambda x: model.encode(x) if x.strip() else np.zeros(model.get_sentence_embedding_dimension())
)

# 🚀 Convert embeddings to NumPy arrays for faster similarity search
activity_embeddings_fr = np.stack(df_activities['activity_embedding'].values)
activity_embeddings_ar = np.stack(df_activities['activity_embedding_ar'].values)

# Step 7: Optimized Matching Function
def find_best_match(row):
    # Compute similarity in bulk
    similarities_fr = cosine_similarity([row['embeddings']], activity_embeddings_fr)[0]
    similarities_ar = cosine_similarity([row['embeddings']], activity_embeddings_ar)[0]

    # Combine all similarities
    all_similarities = np.concatenate((similarities_fr, similarities_ar))

    # Get best match
    best_idx = np.argmax(all_similarities)
    best_similarity = all_similarities[best_idx]

    # Debugging output
    print(f"Best index: {best_idx}, Best similarity: {best_similarity}")

    if best_similarity >= 0.65 and best_idx < len(df_activities):
        best_match = df_activities.iloc[best_idx]
        return best_match['name_activity'], best_match['ar_name_activity'], best_match['code_activity']

    # Ensure we return a tuple of three elements
    return None, None, None

# Check if df_mixed and df_activities are not empty
if not df_mixed.empty and not df_activities.empty:
    # Apply the optimized matching function
    results = df_mixed.apply(find_best_match, axis=1)

    # Ensure results is a DataFrame
    df_results = pd.DataFrame(results.tolist(), columns=['matched_activity', 'matched_activity_ar', 'matched_code'])

    # Assign the results back to df_mixed
    df_mixed[['matched_activity', 'matched_activity_ar', 'matched_code']] = df_results

    # Save results
    df_mixed.to_excel('/content/drive/My Drive/hackathon1/matched_activities.xlsx', index=False)
    print("Matching process completed and saved to Excel.")

    # Filter rows where a match was found
    df_matched = df_mixed[df_mixed['matched_activity'].notna() & df_mixed['matched_code'].notna()]

    # Print matched rows
    print(df_matched[['activity', 'matched_activity', 'matched_activity_ar', 'matched_code']])

    # Save matched rows to a separate file
    matched_file_path = "/content/drive/My Drive/hackathon1/matched_activities_only.xlsx"
    df_matched.to_excel(matched_file_path, index=False)
    print(f"Matched activities saved to: {matched_file_path}")

    # Filter rows where NO match was found
    df_not_matched = df_mixed[df_mixed['matched_activity'].isna() | df_mixed['matched_code'].isna()]

    # Save unmatched rows to a separate file
    not_matched_file_path = "/content/drive/My Drive/hackathon1/not_matched_activities.xlsx"
    df_not_matched.to_excel(not_matched_file_path, index=False)
    print(f"Unmatched activities saved to: {not_matched_file_path}")
else:
    print("No data to process. Please check the input DataFrames.")


Some weights of the model checkpoint at Alibaba-NLP/gte-multilingual-base were not used when initializing NewModel: {'classifier.weight', 'classifier.bias'}
- This IS expected if you are initializing NewModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing NewModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Best index: 2737, Best similarity: 0.7876622764111374
Best index: 1633, Best similarity: 0.6239002080027085
Best index: 1443, Best similarity: 0.6860524415969849
Best index: 1427, Best similarity: 0.9999998807907104
Best index: 2016, Best similarity: 0.7490373053329216
Best index: 1400, Best similarity: 0.6958309412002563
Best index: 1443, Best similarity: 0.9755682349205017
Best index: 1764, Best similarity: 0.6860995067087932
Best index: 920, Best similarity: 0.810370922088623
Best index: 2879, Best similarity: 0.6806698013991368
Best index: 1678, Best similarity: 0.7453273553012101
Best index: 1857, Best similarity: 1.0000000000000009
Best index: 1474, Best similarity: 0.7737803280592315
Best index: 541, Best similarity: 0.7451014518737793
Best index: 2372, Best similarity: 0.7059503140099703
Best index: 1641, Best similarity: 0.7413548577344127
Best index: 263, Best similarity: 0.6754361391067505
Best index: 2893, Best similarity: 0.7714579668549555
Best index: 1171, Best similarit

In [29]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util

# Load a multilingual model for embeddings
model = SentenceTransformer("Alibaba-NLP/gte-multilingual-base", trust_remote_code=True)

# Lists of traditional commercial activities (to exclude)
traditional_commercial_activities = [
    "restaurant", "vente", "achat", "café", "fast food", "boucherie", "poissonnerie", "téléphone",
    "bijouterie", "magasin", "boutique", "fournisseur", "détaillant", "grossiste", "revendeur",
    "importateur", "exportateur", "franchise", "vendeur", "commerçant", "négoce", "dépôt vente",
    "magasin alimentaire", "produits alimentaires", "produits de consommation", "liquidation",
    "bazar", "marché", "vente de voitures", "agents commerciaux", "services de livraison",
    "construction", "matériel de construction", "équipement industriel", "téléphonie mobile", "réparation",
    "location", "distribution", "importation", "commerce de détail", "distributeur", "commerçant de proximité",
    "agricole", "produits agricoles", "alimentation générale", "commerçant ambulant", "transports",
    "مطعم", "بيع", "شراء", "مقهى", "وجبات سريعة", "جزارة", "سمك", "هاتف", "مجوهرات", "متجر", "دكان",
    "مورد", "تاجر تجزئة", "تاجر جملة", "بائع", "مستورد", "مُصدر", "امتياز", "تجارة", "بيع بالتجزئة",
    "منتجات غذائية", "منتجات استهلاكية", "تصفية", "بازار", "سوق", "بيع السيارات", "وكلاء تجاريين",
    "خدمات التوصيل", "إنشاء", "معدات البناء", "تجهيزات صناعية", "هواتف محمولة", "صيانة", "إيجار",
    "توزيع", "استيراد", "تجارة التجزئة", "موزع", "تاجر محلي", "زراعي", "منتجات زراعية", "تجارة عامة",
    "تاجر متجول", "نقل"
]

# Allowed e-commerce & intermediary activities (to keep)
ecommerce_intermediary_activities = [
    "e-commerce", "plateforme de vente", "e-shop", "site marchand", "marketplace", "dropshipping",
    "affiliation", "vente en ligne", "webshop", "commerce virtuel", "commerce digital", "fournisseur e-commerce",
    "commerce électronique", "start-up", "entrepreneur digital", "services de paiement", "agent commercial",
    "courtier", "publicité en ligne", "consultant", "services en ligne", "freelance", "plateforme de freelancing",
    "commercialisation", "digital marketing", "plateforme B2B", "réseau de distribution",
    "تجارة إلكترونية", "منصة بيع", "متجر إلكتروني", "موقع تجاري", "سوق إلكتروني", "دروبشيبينغ",
    "تسويق بالعمولة", "متجر على الإنترنت", "تجارة افتراضية", "تجارة رقمية", "مورد تجارة إلكترونية",
    "شركة ناشئة", "ريادي رقمي", "خدمات الدفع", "وكيل تجاري", "سمسار", "إعلانات عبر الإنترنت",
    "مستشار", "خدمات عبر الإنترنت", "مستقل", "منصة عمل مستقل", "تسويق", "تسويق رقمي", "إدارة الحملات",
    "منصة إعلانات", "تسويق بالعمولة", "وسيط", "وكيل مبيعات", "حلول رقمية", "تخزين سحابي", "دعم العملاء",
    "إعلانات رقمية", "وسائل التواصل الاجتماعي", "منصة بي تو بي", "شبكة توزيع"
]

# Normalize activity lists
traditional_commercial_activities = [normalize_text(act) for act in traditional_commercial_activities]
ecommerce_intermediary_activities = [normalize_text(act) for act in ecommerce_intermediary_activities]

# Encode reference lists
embeddings_commercial = model.encode(traditional_commercial_activities, convert_to_tensor=True)
embeddings_ecommerce = model.encode(ecommerce_intermediary_activities, convert_to_tensor=True)

# Function to determine if an activity is traditional commercial
def classify_activity(activity_text):
    if pd.isna(activity_text):
        return False

    activity_embedding = model.encode(activity_text, convert_to_tensor=True)

    # Compute similarity scores
    similarity_commercial = util.cos_sim(activity_embedding, embeddings_commercial).max().item()
    similarity_ecommerce = util.cos_sim(activity_embedding, embeddings_ecommerce).max().item()

    # Threshold-based filtering
    if similarity_commercial > 0.7 and similarity_commercial > similarity_ecommerce:
        return True  # Exclude (Traditional Commercial)

    return False  # Keep (E-commerce / Valid Activity)

# Apply filtering
df_not_matched['is_traditional_commercial'] = df_not_matched['activity'].apply(classify_activity)

# Separate valid and removed activities
df_valid_activities = df_not_matched[df_not_matched['is_traditional_commercial'] == False].drop(columns=['is_traditional_commercial'])
df_removed_activities = df_not_matched[df_not_matched['is_traditional_commercial'] == True].drop(columns=['is_traditional_commercial'])

# Save filtered datasets
valid_activities_file = "/content/drive/My Drive/hackathon1/final_filtered_non_commercial.xlsx"
removed_activities_file = "/content/drive/My Drive/hackathon1/removed_commercial_activities.xlsx"

df_valid_activities.to_excel(valid_activities_file, index=False)
df_removed_activities.to_excel(removed_activities_file, index=False)

print(f"✅ Final filtered non-commercial & e-commerce activities saved to: {valid_activities_file}")
print(f"❌ Removed traditional commercial activities saved to: {removed_activities_file}")


Some weights of the model checkpoint at Alibaba-NLP/gte-multilingual-base were not used when initializing NewModel: {'classifier.weight', 'classifier.bias'}
- This IS expected if you are initializing NewModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing NewModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
<ipython-input-29-28a044a62975>:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_not_matched['is_traditional_commercial'] = df_not_matched['acti

✅ Final filtered non-commercial & e-commerce activities saved to: /content/drive/My Drive/hackathon1/final_filtered_non_commercial.xlsx
❌ Removed traditional commercial activities saved to: /content/drive/My Drive/hackathon1/removed_commercial_activities.xlsx
